In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 10)

In [2]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

### Tables in the process

In [3]:
# Sells table in MySQL portfolio database
sql = """
SELECT T.name, B.date AS buy_date, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,buy_date,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,IMH,2021-12-08,899,1093,2022-01-10,16.80,52.16,3.65,"25,144.19",33,"8,607.64",52.05%,575.73%,101,None,0


In [4]:
# Buys table in MySQL portfolio database
buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT T.name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT T.name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1093



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,IMH,1093,199,2021-12-08,"1,500",11.00,34.16,2.39,Sold,"16,536.55",DTD,None


In [22]:
# Buys table in MySql portfolio database
sql = """
SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1"""
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1


,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,IP,1094,194,2022-01-10,"3,000",19.20,119.23,8.35,Active,"57,727.58",DTD,None


In [23]:
# Stocks table in MySQL portfolio database
names = buys['name']
name = names.to_string(index=False)

sql = """
SELECT T.* FROM stocks T
WHERE name = '%s'"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conpf)
stocks.style.format(format_dict)


SELECT T.* FROM stocks T
WHERE name = 'IP'


,id,name,category_id,website
0,194,IP,4,


In [24]:
# Buy table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.style.format(format_dict)


SELECT * FROM buy
WHERE name = 'IMH'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,"3,000.0",11.00,0.0,0.0,1,None,3,A3


In [26]:
# Buy table in MySql stock database
names = buys['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.style.format(format_dict)


SELECT * FROM buy
WHERE name = 'IP'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IP,2021-11-09,"7,500.0",20.60,0.0,0.0,1,0.185000,3,A3


In [7]:
# Dividend table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM dividend
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * FROM dividend
WHERE name = 'IMH'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL


### Sell transactions

### Begin of Sale process to buy table in MySQL stock database

In [6]:
# Sells table in MySQL portfolio database
sql = """
SELECT T.name, B.date AS buy_date, B.qty, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,buy_date,qty,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,IMH,2021-12-08,"1,500",899,1093,2022-01-10,16.80,52.16,3.65,"25,144.19",33,"8,607.64",52.05%,575.73%,101,None,0


In [7]:
names = sells['name']
name = names.to_string(index=False)
name

'IMH'

In [8]:
sr_qty = sells['qty']
qty_str = sr_qty.to_string(index=False)
qty = int(qty_str) * -1
qty

-1500

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'IMH'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,4500.0,11.0,0.0,0.0,1,None,3,A3


In [10]:
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE buy
SET volbuy = volbuy + -1500
WHERE name = "IMH"


1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'IMH'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,3000.0,11.0,0.0,0.0,1,None,3,A3


In [15]:
sqlDel = '''
DELETE FROM buy
WHERE volbuy = 0'''
rp = const.execute(sqlDel)
rp.rowcount

0

### End of Sale process to buy table in MySQL stock database

### Begin of Sale process to stocks table in SQLite port_lite database

In [13]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IMH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,684,IMH,16.80,2.08,I,0.00,16.80,158.70,0.33,11.00,"1,500",-12,22,"4,500",0,0,24H,mai


In [17]:
sell_target = 19.00
sqlUpd = '''
UPDATE stocks
SET available_qty = available_qty + %s,
sell_target = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, sell_target, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -1500,
sell_target = 19.0
WHERE name = "IMH"


In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [19]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IMH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,684,IMH,16.80,2.08,I,0.00,19.00,158.70,0.33,11.00,"1,500",-12,22,"3,000",0,0,24H,mai


### End of Sale process to stocks table in SQLite port_lite database

### Begin of Update dividend table in MySQL stock database for sells transactions

In [21]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IMH" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL


In [ ]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

rp = const.execute(sqlDel)
rp.rowcount

### End of Update table dividend in mysql stock database for sells transactions

### Buys transactions

### Begin of Buy process to buy table in MySQL database

In [2]:
sql = """
SELECT * 
FROM buys
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT * 
FROM buys
ORDER BY id DESC
LIMIT 1



,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,1092,113,2022-01-07,6000,31.5,391.23,27.39,Active,189418.62,DTD,None


### In case there is the same stock name in port

In [29]:
buy_quantities = buys['qty']
buy_qty = buy_quantities.to_string(index=False)
buy_qty = int(buy_qty) * 1
buy_qty

3000

In [30]:
new_cost = 20.20 # new average cost must be calculated manually beforehand

sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (buy_qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 3000, price = 20.2
WHERE name = "IP"


In [31]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### In case of totally new stock

In [7]:
sqlIns = """
INSERT INTO buy 
VALUES('SYNEX','2022-01-07',6000,31.5,0,0,1,0,'3','A2')
"""
print(sqlIns)


INSERT INTO buy 
VALUES('SYNEX','2022-01-07',6000,31.5,0,0,1,0,'3','A2')



In [8]:
rp = const.execute(sqlIns)
rp.rowcount

1

### End of Buy process to buy table in MySQL database

### Begin of Buy process to stocks table in SQLite port_lite database

In [32]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,665,IP,24.47,10.66,B,19.20,0.00,6.93,0.76,20.60,"3,000",-8,9,"7,500",0,0,RD05pct,mai


In [34]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, cost = %s, sell_target = 23.20, status = 'I', reason = '15pct'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (buy_qty, new_cost, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 3000, cost = 20.2, sell_target = 23.20, status = 'I', reason = '15pct'
WHERE name = 'IP'



In [35]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [36]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,665,IP,24.47,10.66,I,19.20,23.20,6.93,0.76,20.20,"3,000",-8,9,"10,500",0,0,15pct,mai


### End of Buy process to stocks table in SQLite port_lite database

### Begin of Update dividend table in MySQL stock database for buy transactions

In [37]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IP" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,IP,0.185,0.000,0.000,0.000,0.185,0.00,0.00%,"7,500.0",2022-04-27,2022-04-27,,0


In [38]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (buy_qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 3000
WHERE name = "IP"


In [39]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [40]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IP" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,IP,0.185,0.000,0.000,0.000,0.185,0.00,0.00%,"10,500.0",2022-04-27,2022-04-27,,0


### End of Update dividend table in MySQL stock database for buy transactions